In [1]:
#LOAD DATA
import json
import os
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.data.datasets import register_coco_instances

#def load_data(t="train"):
#    if t == "train":
#        with open("/local/home/stuff/crowdai_buildings_segmentation/train/annotation.json", 'r') as file:
#            train = json.load(file)
#        return train
#    elif t == "val":
#        with open("/local/home/stuff/crowdai_buildings_segmentation/val/annotation.json", 'r') as file:
#            val = json.load(file)
#        return val

# Register datasets
from detectron2.data.datasets import register_coco_instances
#for d in ["train", "val"]:
register_coco_instances("segmentation_train", {}, "/local/home/stuff/datasets/crowdai/train/annotation.json", "/local/home/stuff/datasets/crowdai/train/images/")
register_coco_instances("segmentation_val", {}, "/local/home/stuff/datasets/crowdai/val/annotation.json", "/local/home/stuff/datasets/crowdai/val/images/")
MetadataCatalog.get("segmentation_train").set(thing_classes=["building"])
MetadataCatalog.get("segmentation_val").set(thing_classes=["building"])

# Check if all annotations in the registered training set have the segmentation field
from detectron2.data import DatasetCatalog

dataset_dicts_train = DatasetCatalog.get('segmentation_train')

for d in dataset_dicts_train:
    for obj in d['annotations']:
        if 'bbox' not in obj:
            print(f'{d["file_name"]} has an annotation with no bbox field')
            
#for d in ["train", "val"]:
    #DatasetCatalog.register(d, lambda d=d: load_data(d))
    #register_coco_instances(d, {}, "/local/home/stuff/crowdai_buildings_segmentation/" + d + "/annotation.json", "/local/home/stuff/crowdai_buildings_segmentation/" + d +"/images/")
    #MetadataCatalog.get(d).set(thing_classes=["building"])
#metadata = MetadataCatalog.get("train")

/local/home/bsoyuer/miniconda3/envs/detectron/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm

Category ids in annotations are not in [1, #categories]! We'll apply a mapping for you.



In [2]:
import numpy as np
temp = []
for d in dataset_dicts_train:
    for obj in d['annotations']:
        temp.append(obj['category_id'])
print(np.unique(np.array(temp)))

[0]


In [22]:
#CONFIGURE MODEL
from detectron2.config import get_cfg
from detectron2 import model_zoo


def custom_config(num_classes):
    
    cfg = get_cfg()
    
    # get configuration from model_zoo
    cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
    cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")

    # Model
    cfg.MODEL.MASK_ON = True
    cfg.MODEL.ROI_HEADS.NUM_CLASSES = num_classes
    cfg.MODEL.BACKBONE.NAME = "build_resnet_backbone"
    cfg.MODEL.RESNETS.DEPTH = 18
    cfg.MODEL.RESNETS.RES2_OUT_CHANNELS = 64
    cfg.MODEL.RPN.IN_FEATURES = ['p2', 'p3', 'p4', 'p5']
    
    
    # Solver
    cfg.SOLVER.BASE_LR = 0.0002
    cfg.SOLVER.MAX_ITER = 40000
    cfg.SOLVER.STEPS = (20, 10000, 20000)
    cfg.SOLVER.gamma = 0.5
    cfg.SOLVER.IMS_PER_BATCH = 4
    
    # Test
    cfg.TEST.DETECTIONS_PER_IMAGE = 20
    
    # INPUT
    #cfg.INPUT.MIN_SIZE_TRAIN = (300,300)
    
    # DATASETS
    cfg.DATASETS.TEST = ('segmentation_val',)
    cfg.DATASETS.TRAIN = ('segmentation_train',)
    
    # DATASETS
    cfg.OUTPUT_DIR = "/local/home/bsoyuer/citiies_contrastive_learning/detectron2"
    
    return cfg

In [19]:
print(custom_config(1))

CUDNN_BENCHMARK: False
DATALOADER:
  ASPECT_RATIO_GROUPING: True
  FILTER_EMPTY_ANNOTATIONS: True
  NUM_WORKERS: 4
  REPEAT_THRESHOLD: 0.0
  SAMPLER_TRAIN: TrainingSampler
DATASETS:
  PRECOMPUTED_PROPOSAL_TOPK_TEST: 1000
  PRECOMPUTED_PROPOSAL_TOPK_TRAIN: 2000
  PROPOSAL_FILES_TEST: ()
  PROPOSAL_FILES_TRAIN: ()
  TEST: ('segmentation_val',)
  TRAIN: ('segmentation_train',)
GLOBAL:
  HACK: 1.0
INPUT:
  CROP:
    ENABLED: False
    SIZE: [0.9, 0.9]
    TYPE: relative_range
  FORMAT: BGR
  MASK_FORMAT: polygon
  MAX_SIZE_TEST: 1333
  MAX_SIZE_TRAIN: 1333
  MIN_SIZE_TEST: 800
  MIN_SIZE_TRAIN: (640, 672, 704, 736, 768, 800)
  MIN_SIZE_TRAIN_SAMPLING: choice
  RANDOM_FLIP: horizontal
MODEL:
  ANCHOR_GENERATOR:
    ANGLES: [[-90, 0, 90]]
    ASPECT_RATIOS: [[0.5, 1.0, 2.0]]
    NAME: DefaultAnchorGenerator
    OFFSET: 0.0
    SIZES: [[32], [64], [128], [256], [512]]
  BACKBONE:
    FREEZE_AT: 2
    NAME: build_resnet_backbone
  DEVICE: cuda
  FPN:
    FUSE_TYPE: sum
    IN_FEATURES: ['res2'

In [21]:
#TRAIN
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.engine import DefaultTrainer

  
#if __name__ == '__main__':
    
#for d in ["train", "val"]:
        
    #DatasetCatalog.register(d, lambda d=d: load_data(d))
    #register_coco_instances(d, {}, "/local/home/stuff/crowdai_buildings_segmentation/" + d + "/annotation.json", "/local/home/stuff/crowdai_buildings_segmentation/" + d +"/images/")
    #MetadataCatalog.get(d).set(thing_classes=["Building"])
        
metadata = MetadataCatalog.get("segmentation_train")
MetadataCatalog.get("segmentation_train").set(thing_classes=["building"])

cfg = custom_config(1)

trainer = DefaultTrainer(cfg)
trainer.resume_or_load(resume=False)
trainer.train()

KeyError: 'p2'

In [ ]:
#VISUALIZE
from detectron2.engine import DefaultPredictor
from detectron2.utils.visualizer import Visualizer, ColorMode
import matplotlib.pyplot as plt
import cv2
import os


def visualization(metadata, cfg, test_set):
    cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
    cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.8
    predictor = DefaultPredictor(cfg)
    for d in test_set:
        im = cv2.imread(d["file_name"])
        outputs = predictor(
            im)
        v = Visualizer(im[:, :, ::-1],
                       metadata=metadata,
                       scale=0.5,
                       instance_mode=ColorMode.IMAGE_BW
                       )
        out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
        img = cv2.cvtColor(out.get_image()[:, :, ::-1], cv2.COLOR_RGBA2RGB)
        plt.imsave(os.path.join(os.path.join(cfg.OUTPUT_DIR, 'visualization'), str(d["image_id"]) + '.png'), img)